# Telecom_DC

Клетка для необходимых включений

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import numpy as np
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [82]:
df = pd.read_csv("../data/train/base_train.csv", sep=';')

In [83]:
# target = df[['sk_id', 'csi']]
df.drop(['csi'], axis=1, inplace=True)

In [84]:
df.head()

,sk_id,cell_lac_id,data_mb,data_min,speed,voice_min,data_mb_m,number_dat,number_voi,voice,g2,g3,g4,internet_type
0,3564,NaN,0.000000,0.000000,0.000000,0.001071,NaN,NaN,2.0,0.023958,1.0,1.0,0.0,2.0
1,919,2.0,0.000003,0.000641,0.005134,0.002197,0.000032,2.0,2.0,0.053125,1.0,1.0,0.0,2.0
2,1903,NaN,0.000000,0.000000,0.000000,0.000155,NaN,NaN,1.0,0.003681,1.0,1.0,0.0,2.0
3,2111,2.0,0.000001,0.000467,0.002592,0.000321,0.000013,2.0,1.0,0.007708,1.0,1.0,0.0,1.0
4,766,3.0,0.000076,0.002504,0.030249,0.000000,0.000777,7.0,NaN,NaN,1.0,1.0,1.0,1.0


In [85]:
df['new_feature'] = df['number_dat'] * df['speed']

In [86]:
df['sot_is_3g_but_phone_is_4g'] = (df['g4'] == 0) & (df['internet_type'] == 1.0)

In [78]:
df

,sk_id,g4,voice_min,internet_type,voice,speed,data_mb_m,number_voi,number_dat,g3,g2,data_min,sot_is_3g_but_phone_is_4g,new_feature
0,2,0.0,0.093695,2.0,1.702153,0.000000,6.722706e-02,131.0,1020.0,1.0,1.0,1.067549e+00,False,0.000000
1,3,0.0,0.111219,2.0,2.103264,0.000000,5.999787e-02,246.0,919.0,1.0,1.0,1.347779e+00,False,0.000000
2,5,0.0,0.388235,2.0,6.409166,0.000000,3.142074e+00,849.0,2957.0,0.0,1.0,2.265131e+00,False,0.000000
3,7,0.0,0.243886,1.0,4.496666,0.000000,8.558853e-01,398.0,2250.0,1.0,1.0,2.778798e+00,True,0.000000
4,9,0.0,0.006056,2.0,0.146250,0.000000,3.642922e-05,8.0,4.0,1.0,1.0,2.761000e-05,False,0.000000
5,11,0.0,0.003077,1.0,0.000000,0.000000,6.307415e-02,0.0,819.0,1.0,1.0,3.823098e-01,True,0.000000
6,12,0.0,0.313724,2.0,5.127569,0.000000,5.000511e-01,460.0,1617.0,0.0,1.0,1.937425e+00,False,0.000000
7,14,0.0,0.196233,2.0,3.350277,0.000000,6.756472e-06,252.0,82.0,1.0,1.0,9.913800e-04,False,0.000000
8,15,0.0,0.148154,2.0,2.473680,0.002652,9.311522e-02,216.0,689.0,0.0,1.0,1.009683e+00,False,1.827115
9,16,0.0,0.033268,1.0,0.356667,0.004830,1.310755e-01,27.0,199.0,0.0,1.0,1.934183e-01,True,0.961194


In [72]:
df['internet_type'] == 1.0

0       False
1       False
2       False
3        True
4       False
5        True
6       False
7       False
8       False
9        True
10      False
11      False
12       True
13      False
14       True
15      False
16       True
17      False
18       True
19      False
20      False
21      False
22      False
23       True
24      False
25       True
26      False
27      False
28       True
29       True
        ...  
2712    False
2713    False
2714    False
2715     True
2716    False
2717     True
2718    False
2719    False
2720    False
2721    False
2722     True
2723     True
2724     True
2725    False
2726    False
2727     True
2728    False
2729    False
2730    False
2731    False
2732    False
2733     True
2734    False
2735    False
2736     True
2737    False
2738    False
2739     True
2740     True
2741    False
Name: internet_type, Length: 2742, dtype: bool

In [33]:
target.shape

(725502, 2)

In [90]:
def groupby(df, features_functions):
    df = df.groupby(['cell_lac_id','sk_id']).agg(features_functions)

    df = df.reset_index(level=['cell_lac_id', 'sk_id'])

    df = df.groupby(['sk_id']).agg(features_functions)
    df = df.reset_index(level=['sk_id'])
    
    return df

In [88]:
df.head()

,sk_id,cell_lac_id,data_mb,data_min,speed,voice_min,data_mb_m,number_dat,number_voi,voice,g2,g3,g4,internet_type,new_feature,sot_is_3g_but_phone_is_4g
0,3564,NaN,0.000000,0.000000,0.000000,0.001071,NaN,NaN,2.0,0.023958,1.0,1.0,0.0,2.0,NaN,False
1,919,2.0,0.000003,0.000641,0.005134,0.002197,0.000032,2.0,2.0,0.053125,1.0,1.0,0.0,2.0,0.010268,False
2,1903,NaN,0.000000,0.000000,0.000000,0.000155,NaN,NaN,1.0,0.003681,1.0,1.0,0.0,2.0,NaN,False
3,2111,2.0,0.000001,0.000467,0.002592,0.000321,0.000013,2.0,1.0,0.007708,1.0,1.0,0.0,1.0,0.005183,True
4,766,3.0,0.000076,0.002504,0.030249,0.000000,0.000777,7.0,NaN,NaN,1.0,1.0,1.0,1.0,0.211743,False


In [92]:
features_functions = {'speed': np.mean,
                        'data_mb_m':np.sum,
                        'voice_min':np.sum,
                        'data_min':np.sum,
                        'number_dat':np.sum,
                        'number_voi':np.sum,
                        'voice': np.sum,
                        'g2': np.min,
                        'g3': np.min,
                        'g4': np.min,
                        'internet_type': np.min,
                        'new_feature':np.mean,
                         'sot_is_3g_but_phone_is_4g':np.min}
df = groupby(df, features_functions)

In [66]:
df = df.groupby(['cell_lac_id','sk_id']).agg({'speed': np.mean,
                                              'data_mb_m':np.sum,
                                              'voice_min':np.sum,
                                              'data_min':np.sum,
                                              'number_dat':np.sum,
                                              'number_voi':np.sum,
                                              'voice': np.sum,
                                              'g2': np.min,
                                              'g3': np.min,
                                              'g4': np.min,
                                              'internet_type': np.min})

df = df.reset_index(level=['cell_lac_id', 'sk_id'])

df = df.groupby(['sk_id']).agg({'speed': np.min,
                                              'data_mb_m':np.sum,
                                              'voice_min':np.sum,
                                              'data_min':np.sum,
                                              'number_dat':np.sum,
                                              'number_voi':np.sum,
                                              'voice': np.sum,
                                              'g2': np.min,
                                              'g3': np.min,
                                              'g4': np.min,
                                              'internet_type': np.min})
df = df.reset_index(level=['sk_id'])

In [93]:
df_new = pd.merge(df, data_base, left_on='sk_id', right_on='SK_ID', how='left')
df_new.drop(['sk_id', 'SK_ID'], axis=1, inplace=True)

In [129]:
param = {'objective':'binary:logistic', 'n_estimators':120, 'max_depth': 2, 'min_split_loss':4, 
         'learning_rate':0.1}
model_boost = xgb.XGBModel(**param)

skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=0)

counter = 0


param_grid = {'n_estimaros':[100,40,20]}

X = df_new.drop(['CSI'], axis=1)
y =  df_new['CSI']

test_roc_auc = []
train_roc_auc = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model_boost.fit(X_train,y_train,eval_metric='auc',verbose=True)
    pred = model_boost.predict(X_test)
    test_roc_auc.append(roc_auc_score(y_test,pred))
    train_roc_auc.append(roc_auc_score(y_train, model_boost.predict(X_train)))

print("av. train error:", np.mean(np.array(train_roc_auc)))
print("av. test error:", np.mean(np.array(test_roc_auc)))

av. train error: 0.6735870114050178
av. test error: 0.5141742816950924


In [105]:
np.mean(np.asarray(ra_boost))

0.5106731063727872

In [130]:
print(X.columns)
model_boost.feature_importances_

Index(['number_voi', 'data_mb_m', 'sot_is_3g_but_phone_is_4g', 'g3',
       'internet_type', 'g4', 'voice_min', 'new_feature', 'g2', 'speed',
       'voice', 'number_dat', 'data_min'],
      dtype='object')


array([0.14814815, 0.16666667, 0.        , 0.        , 0.07407407,
       0.        , 0.01851852, 0.07407407, 0.        , 0.2962963 ,
       0.12962963, 0.01851852, 0.07407407], dtype=float32)

In [ ]:
data_cons = data_cons.groupby(['SK_ID']).agg({'SUM_MINUTES':np.sum,
                                                 'SUM_DATA_MB':np.sum,
                                                 'SUM_DATA_MIN':np.sum,'SPEED':np.mean})
data_cons = data_cons.reset_index(level=['SK_ID'])
data_cons[20:40]

In [5]:
!pip3 install xlrd>=0.9.0

You are using pip version 8.1.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
df = pd.read_excel

In [20]:

def list_to_float(data,list_):
    temp_data = data
    for el in list_:
        temp_data[el] = to_float(temp_data,el)
    return temp_data

# Вспомогательная функция
def to_float(data,col):
    if not data[col].dtype == np.float64:
        return data[col].apply(lambda x: float('0.'+str(x[1:])))
    else:
        return data[col]


def calculate_means(numeric_data):
    means = np.zeros(numeric_data.shape[1])
    for j in range(numeric_data.shape[1]):
        to_sum = numeric_data.iloc[:,j]
        indices = np.nonzero(~numeric_data.iloc[:,j].isnull())[0]
        correction = np.amax(to_sum[indices])
        to_sum /= correction
        for i in indices:
            means[j] += to_sum[i]
        means[j] /= indices.size
        means[j] *= correction
    return pd.Series(means, numeric_data.columns)




def plot_roc_auc(y_test,pred):
    fpr, tpr, _ = roc_curve(y_test, pred)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
     lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

def transoform(row):
    if row['SUM_DATA_MIN'] != 0:
        return row['SUM_DATA_MB']/row['SUM_DATA_MIN']
    else:
        return 0

In [2]:
import os
os.environ['KERAS_BACKEND'] = 'theano'
import keras as ks

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils

In [36]:
train_data_path = '../data/train/'

def get_train_data(path):
    return train_data_path + path

In [5]:
def test_xgboost(data):

    param = {'objective':'binary:logistic', 'n_estimators':20}

    model_boost = xgb.XGBModel(**param)
# Or you can use: clf = xgb.XGBClassifier(**param_dist)
    X = data.drop(['CSI','SK_ID'],axis=1)
    y = data['CSI']
    corr = []
    skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=0)
    ra_boost = []
    counter = 0
    param_grid = {'n_estimaros':[100,40,20]}
    for train_index, test_index in skf.split(X, y):

     
        
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = y[train_index], y[test_index]
#        rocauc = roc_auc_score(pred,y_test)
#        print(rocauc)
        model_boost.fit(X_train,y_train,eval_metric='auc',verbose=True)
        pred = model_boost.predict(X_test)
        ra_boost.append(roc_auc_score(y_test,pred))
        predictions = model_boost.predict(X_site)
#        print('{} - TEST, {} - correlation'.format(roc_auc_score(y_test,pred),roc_auc_score(best,predictions)))
#    print('{} - correlation'.format(roc_auc_score(best,predictions)))
    corr.append(roc_auc_score(best,predictions))
    #print(ra_boost)
#    print('{} - average rocauc'.format(sum(ra_boost, 0.0) / len(ra_boost)))
#    plot_roc_auc(y_test,model_boost.predict(X_test))
#    print(model_boost.feature_importances_)
    print(sum(corr, 0.0)/ len(corr))
    return model_boost

## Подключение основной таблицы csi_train

In [37]:
data_base = pd.read_csv(get_train_data('subs_csi_train.csv'),sep=';')
data_base = data_base.drop(['CONTACT_DATE'],axis=1)
data_base.head()

,SK_ID,CSI
0,2,0
1,3,0
2,4,0
3,5,0
4,6,1


In [38]:
data_base.shape

(4222, 2)

Данных явно недостаточно для того чтобы сделать хоть какое-нибудь предсказание. Расммотрим следующую таблицу

##  Подключение таблицы subs_bs_voice_session

In [8]:
data_voice_session = pd.read_csv(get_train_data('subs_bs_voice_session_train.csv'),sep=';')

In [9]:
data_voice_session['VOICE_DUR_MIN']=data_voice_session['VOICE_DUR_MIN'].apply(lambda x: float('0.'+str(x[1:])))
data_voice_session = data_voice_session.groupby(['CELL_LAC_ID','SK_ID']).agg({'VOICE_DUR_MIN': np.sum})
data_voice_session = data_voice_session.reset_index(level=['CELL_LAC_ID', 'SK_ID'])
data_voice_session.head()

,CELL_LAC_ID,SK_ID,VOICE_DUR_MIN
0,1,3564,0.023958
1,2,919,0.053125
2,2,1903,0.003681
3,2,2111,0.007708
4,4,1837,0.160069


## Подключение таблицы consumptions

In [55]:
data_cons = pd.read_csv(get_train_data('subs_bs_consumption_train.csv'),sep=';')
print(data_cons.shape[0])
# cell_features = pd.read_csv('../data/bs_avg/CELL_AVAILABILITY_features.csv')
# data_cons = pd.merge(data_cons, cell_features, on=['CELL_LAC_ID'], how='left')

1092544


In [24]:
data_cons.head()

,SK_ID,CELL_LAC_ID,MON,SUM_MINUTES,SUM_DATA_MB,SUM_DATA_MIN,CELL_AVAILABILITY_2G,CELL_AVAILABILITY_3G
0,1827,240277,1.03,0,",00000243",",00023357",1.0,NaN
1,1827,355195,1.03,",0000716232",0,0,1.0,NaN
2,1827,238554,1.04,0,",00001885",",00032602",1.0,NaN
3,1827,350978,1.04,",0165764688",",00225979",",00432901",1.0,NaN
4,1827,315010,1.04,",0127546562",",00118573",",00249134",1.0,NaN


In [56]:
data_cons.shape

(1092544, 6)

In [66]:

data_cons= list_to_float(data_cons, ['SUM_MINUTES','SUM_DATA_MB','SUM_DATA_MIN'])
data_cons = data_cons.groupby(['CELL_LAC_ID','SK_ID']).agg({'SUM_MINUTES':np.sum,
                                                            'SUM_DATA_MB':np.sum,
                                                            'SUM_DATA_MIN':np.sum})
#                                                             'CELL_AVAILABILITY_2G': np.nanmax,
#                                                             'CELL_AVAILABILITY_3G': np.nanmax})
# data_cons = data_cons.fillna(0)
data_cons = data_cons.reset_index(level=['CELL_LAC_ID', 'SK_ID'])
data_cons['SPEED'] = data_cons.apply(lambda row: transoform(row),axis=1)


data_cons = data_cons.groupby(['SK_ID']).agg({'SUM_MINUTES':np.sum,
                                                 'SUM_DATA_MB':np.sum,
                                                 'SUM_DATA_MIN':np.sum,'SPEED':np.mean})
data_cons = data_cons.reset_index(level=['SK_ID'])
data_cons[20:40]

,SK_ID,SPEED,SUM_DATA_MB,SUM_DATA_MIN,SUM_MINUTES
20,22,0.001447,0.011387,1.095266,0.387132
21,23,0.007919,0.000004,0.000046,0.053921
22,24,0.026343,0.079960,2.980037,0.131351
23,25,0.108965,0.086934,2.521596,0.193019
24,26,0.000000,0.000000,0.000000,0.091638
25,27,0.000000,0.000000,0.000000,0.003593
26,28,0.045247,0.090305,1.842172,0.202327
27,30,0.021082,0.175517,2.105845,0.228111
28,31,0.000000,0.000000,0.000000,0.128297
29,33,0.000000,0.000000,0.000000,0.119104


In [67]:
data_cons.shape[0]

4215

## Поключение sub_data_session

In [62]:
data_data_session = pd.read_csv(get_train_data('subs_bs_data_session_train.csv'),sep=';')
data_data_session['DATA_VOL_MB'] = to_float(data_data_session, 'DATA_VOL_MB')
data_data_session = data_data_session.groupby(['CELL_LAC_ID','SK_ID']).agg({'DATA_VOL_MB': np.sum})
data_data_session = data_data_session.reset_index(level=['CELL_LAC_ID', 'SK_ID'])
data_data_session = data_data_session.groupby('SK_ID').agg({'DATA_VOL_MB':np.mean})
data_data_session = data_data_session.reset_index(level=['SK_ID'])

data_data_session.head()


,SK_ID,DATA_VOL_MB
0,2,0.000731
1,3,0.004615
2,5,0.015402
3,7,0.003424
4,9,0.000009


## Подключение sub_features

In [63]:
data_sub_feat = pd.read_csv(get_train_data('subs_features_train.csv'),sep=';')

In [28]:
data_sub_feat.head()

,SNAP_DATE,COM_CAT#1,SK_ID,COM_CAT#2,COM_CAT#3,BASE_TYPE,ACT,ARPU_GROUP,COM_CAT#7,COM_CAT#8,...,COM_CAT#25,COM_CAT#26,COM_CAT#27,COM_CAT#28,COM_CAT#29,COM_CAT#30,COM_CAT#31,COM_CAT#32,COM_CAT#33,COM_CAT#34
0,01.08.01,4,3339,80,5,2,1,12.0,1,1489.0,...,1,1,",031696",",076724",",040279",",0133",",083382",",027579",",103896",2.0
1,01.08.01,4,238,75,5,2,1,7.0,1,945.0,...,1,1,",015893",",012981",0,0,",04893",",027579",",103896",3.0
2,01.08.01,4,4669,43,5,2,1,3.0,3,1033.0,...,1,1,",000415",0,0,0,",009397",",027579",",103896",3.0
3,01.08.01,4,2555,54,5,2,1,12.0,1,512.0,...,1,1,",04651",",039486",",000829",0,",196676",",027579",",103896",1.0
4,01.08.01,4,2291,38,5,2,1,7.0,1,1268.0,...,1,1,",005393",",007385",",003884",0,",047297",",027579",",103896",6.0


In [29]:
data_sub_feat = data_sub_feat.drop(['COM_CAT#24'],axis=1)

list_float = ['COM_CAT#'+str(i) for i in [19,20,21,22,23,31,32,33,27,28,29,30,18,17]]
list_float.extend(['COST','REVENUE','ITC','VAS','RENT_CHANNEL','ROAM'])

data_sub_feat = list_to_float(data_sub_feat, list_float)

data_sub_feat = data_sub_feat.groupby(['SNAP_DATE','SK_ID']).agg({'COM_CAT#20':np.sum,
                                                 'COM_CAT#22':np.sum,
                                                 'COM_CAT#23':np.sum,
                   
                                                 'COM_CAT#31':np.sum,
                                                 
                                                 'COM_CAT#33':np.sum,
                                            'COM_CAT#1':np.sum,
                                          'COM_CAT#7':np.sum,                            
                                          'COM_CAT#8':np.sum,
                                         'COM_CAT#20':np.sum,
                                            'ROAM':np.sum,
                                        'INTERNET_TYPE_ID':np.mean})
data_sub_feat = data_sub_feat.reset_index(level=['SK_ID','SNAP_DATE'])
data_sub_feat = data_sub_feat[data_sub_feat['SNAP_DATE'] == max(data_sub_feat['SNAP_DATE'])]
data_sub_feat = data_sub_feat.drop(['SNAP_DATE'],axis=1)

## Пользовался ли юзер "плохой" клеткой?

In [254]:
data = pd.merge(data_base,data_cons,on='SK_ID',how='left')
unique, counts = np.unique(data[data['CSI']==1].values[:,2],return_counts=True)

In [266]:
data.head()

,SK_ID,CSI,CELL_LAC_ID,SUM_DATA_MB,SUM_DATA_MIN,SUM_MINUTES,SPEED
0,2,0,974.0,0.000006,0.000233,0.000000,0.027299
1,2,0,3532.0,0.000002,0.000234,0.000000,0.006679
2,2,0,3885.0,0.000004,0.000234,0.000132,0.018924
3,2,0,4892.0,0.000000,0.000000,0.000046,0.000000
4,2,0,11466.0,0.000015,0.002707,0.000009,0.005600


In [267]:
a = dict(zip(unique, counts))

In [83]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 15

In [268]:
new_dict = {'CELL_LAC_ID':list(a.keys()),'NUM':list(a.values())}
data_cell = pd.DataFrame.from_dict(new_dict)

In [269]:
data_cell.head()

,CELL_LAC_ID,NUM
0,NaN,1
1,262145.0,4
2,2.0,1
3,262147.0,2
4,5.0,1


In [259]:
voice_test = pd.read_csv(get_test_data('subs_bs_voice_session_test.csv'), sep=';')


In [261]:
merged = pd.merge(voice_test, data_cell, on='CELL_LAC_ID', how='left')

In [264]:
merged['NUM'].isna().sum()/merged['NUM'].shape[0]

0.7362600226643446

In [234]:
merged = pd.merge(test_data, data_voice_session, on='SK_ID', how='left')

## Обединение таблицы

In [64]:
gen_dat_cons = pd.merge(data_base,data_cons,on='SK_ID',how='left')
# gen_dat_cons
#gen_dat_cons = gen_dat_cons.fillna(calculate_means(gen_dat_cons))

gen_dat = pd.merge(gen_dat_cons,data_data_session,on='SK_ID',how='left')
# gen_dat = pd.merge(gen_dat, data_cell, on='CELL_LAC_ID', how='right')
X = gen_dat.drop(['SK_ID','CSI'],axis=1).fillna(0)
y = gen_dat['CSI']

scaler = StandardScaler()
X = scaler.fit_transform(X)
gen_dat = gen_dat.fillna(0)
# X

#gen_dat_voice = pd.merge(data_base,data_voice_session,on='SK_ID',how='left')
#gen_dat_voice = gen_dat_voice.fillna(calculate_means(gen_dat_voice))
#gen_dat_voice

#gen_dat_dat = pd.merge(data_base,data_data_session, on='SK_ID',how='left')
#gen_dat_dat = gen_dat_dat.fillna(calculate_means(gen_dat_dat))
#gen_dat_dat

#gen_dat_feat = pd.merge(data_base, data_sub_feat,on='SK_ID',how='left')
#gen_dat_feat = gen_dat_feat.fillna(calculate_means(gen_dat_feat))

#gen_dat_feat
#X = gen_dat_feat.drop(['CSI','SK_ID'],axis=1)
#y = gen_dat_feat['CSI']


In [65]:
gen_dat.shape

(760452, 8)

# Построение модели

###  XGBoost

In [34]:
gen_m = test_xgboost(gen_dat)
gen_m.feature_importances_

NameError: name 'X_site' is not defined

In [20]:
model = RandomForestClassifier()
#pred1 = cons_m.predict(gen_dat_cons.drop(['CSI','SK_ID'],axis=1))
gen_dat_cons

NameError: name 'RandomForestClassifier' is not defined

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,
                                                    random_state=0)
# Scale the variables to have 0 mean and unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [12]:
from deslib.dcs.a_priori import APriori
from deslib.dcs.mcb import MCB
from deslib.dcs.ola import OLA
from deslib.des.des_p import DESP
from deslib.des.knora_e import KNORAE
from deslib.des.knora_u import KNORAU
from deslib.des.meta_des import METADES
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,
                                                    random_state=0)

# Scale the variables to have 0 mean and unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Split the data into training and DSEL for DS techniques
X_train, X_dsel, y_train, y_dsel = train_test_split(X_train, y_train,
                                                    test_size=0.5,
                                                    random_state=0)

pool_classifiers = RandomForestClassifier(n_estimators=100, random_state=0)
pool_classifiers.fit(X_train, y_train)

# Initialize the DS techniques
knorau = KNORAU(pool_classifiers)
kne = KNORAE(pool_classifiers)
desp = DESP(pool_classifiers)
ola = OLA(pool_classifiers)
mcb = MCB(pool_classifiers)

calibrated_pool = []
for clf in pool_classifiers:
    calibrated = CalibratedClassifierCV(base_estimator=clf, cv='prefit')
    calibrated.fit(X_dsel, y_dsel)
    calibrated_pool.append(calibrated)

apriori = APriori(calibrated_pool)
meta = METADES(calibrated_pool)


knorau.fit(X_dsel, y_dsel,)
kne.fit(X_dsel, y_dsel)
desp.fit(X_dsel, y_dsel)
ola.fit(X_dsel, y_dsel)
mcb.fit(X_dsel, y_dsel)
apriori.fit(X_dsel, y_dsel)


In [16]:
print('Evaluating DS techniques:')
print('Classification accuracy KNORA-Union: ',
      knorau.score(X_test, y_test))
print('Classification accuracy KNORA-Eliminate: ',
      kne.score(X_test, y_test))
print('Classification accuracy DESP: ', roc_auc_score(desp.predict(X_test), y_test))
print('Classification accuracy OLA: ', ola.score(X_test, y_test))
print('Classification accuracy A priori: ', apriori.score(X_test, y_test))
print('Classification accuracy MCB: ', mcb.score(X_test, y_test))
print('Classification accuracy Bagging: ',
      pool_classifiers.score(X_test, y_test))

Evaluating DS techniques:
Classification accuracy KNORA-Union:  0.8228120516499282
Classification accuracy KNORA-Eliminate:  0.7840746054519369
Classification accuracy DESP:  0.4807894238548534
Classification accuracy OLA:  0.7682926829268293
Classification accuracy A priori:  0.8464849354375896
Classification accuracy MCB:  0.7309899569583931
Classification accuracy Bagging:  0.8185078909612625


# Nn

In [36]:
from keras import optimizers

In [51]:
model = Sequential()
model.add(Dense(200, input_shape=(8,), kernel_initializer='normal'))
# model.add(Dense(16, input_shape=(4,)))
model.add(Activation('sigmoid'))


model.add(Dense(100, kernel_initializer='normal'))
model.add(Activation('relu'))
model.add(Dense(40, kernel_initializer='normal'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Activation('sigmoid'))

sgd = optimizers.SGD(lr=0.000001, decay=1e-9, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd, loss='binary_crossentropy')

In [49]:
data  = gen_dat

In [52]:
model.fit(X_train,y_train, epochs=50, batch_size=10000)

Epoch 1/50
509502/509502 [==============================] - 17s 33us/step - loss: 0.6850
Epoch 2/50
509502/509502 [==============================] - 17s 33us/step - loss: 0.6849
Epoch 3/50
509502/509502 [==============================] - 17s 33us/step - loss: 0.6847
Epoch 4/50
509502/509502 [==============================] - 17s 32us/step - loss: 0.6846
Epoch 5/50
509502/509502 [==============================] - 21s 41us/step - loss: 0.6844
Epoch 6/50
509502/509502 [==============================] - 19s 37us/step - loss: 0.6842
Epoch 7/50
509502/509502 [==============================] - 17s 34us/step - loss: 0.6841
Epoch 8/50
509502/509502 [==============================] - 17s 34us/step - loss: 0.6839
Epoch 9/50
509502/509502 [==============================] - 18s 35us/step - loss: 0.6838
Epoch 10/50
509502/509502 [==============================] - 18s 36us/step - loss: 0.6836
Epoch 11/50
509502/509502 [==============================] - 21s 41us/step - loss: 0.6835
Epoch 12/50
509502/

KeyboardInterrupt: 

In [85]:
?model.fit

In [187]:
roc_auc_score(y_test, model.predict_proba(X_test))

0.563528236292464

In [140]:
from keras.layers import Dropout

In [186]:
print('Building Neural Network model...')
adam = optimizers.adam(lr = 0.005, decay = 0.0000001)

model = Sequential()
model.add(Dense(58, input_dim=5,
                kernel_initializer='normal',
                #kernel_regularizer=regularizers.l2(0.02),
                activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(34,
                #kernel_regularizer=regularizers.l2(0.02),
                activation="tanh"))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy", optimizer='adam')
# model.fit(X_train,y_train, epochs=50, batch_size=10)
history = model.fit(X_train, y_train, epochs=70, batch_size=164)

Building Neural Network model...
Epoch 1/70
3167/3167 [==============================] - 0s 54us/step - loss: 0.6349
Epoch 2/70
3167/3167 [==============================] - 0s 76us/step - loss: 0.4902
Epoch 3/70
3167/3167 [==============================] - 0s 55us/step - loss: 0.4555
Epoch 4/70
3167/3167 [==============================] - 0s 54us/step - loss: 0.4403
Epoch 5/70
3167/3167 [==============================] - 0s 55us/step - loss: 0.4416
Epoch 6/70
3167/3167 [==============================] - 0s 58us/step - loss: 0.4356
Epoch 7/70
3167/3167 [==============================] - 0s 58us/step - loss: 0.4370
Epoch 8/70
3167/3167 [==============================] - 0s 56us/step - loss: 0.4323
Epoch 9/70
3167/3167 [==============================] - 0s 58us/step - loss: 0.4342
Epoch 10/70
3167/3167 [==============================] - 0s 54us/step - loss: 0.4365
Epoch 11/70
3167/3167 [==============================] - 0s 65us/step - loss: 0.4354
Epoch 12/70
3167/3167 [==================

In [80]:
X = data.drop(['CSI','SK_ID'],axis=1)
y = data['CSI']
corr = []
skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=0)
#     ra_boost = []
#     counter = 0
#     param_grid = {'n_estimaros':[100,40,20]}
ar = []
# for train_index, test_index in skf.split(X, y):



#     X_train, X_test = X.values[train_index], X.values[test_index]
#     y_train, y_test = y[train_index], y[test_index]
# #        rocauc = roc_auc_score(pred,y_test)
# #        print(rocauc)
#     model.fit(X_train,y_train)
#     pred = model.predict(X_test)
#     ar.append(roc_auc_score(y_test,pred))
# #     predictions = model_boost.predict(X_site)

In [188]:

#     param = {'objective':'binary:logistic', 'n_estimators':20}

#     model_boost = xgb.XGBModel(**param)
# Or you can use: clf = xgb.XGBClassifier(**param_dist)
data = gen_dat
X = data.drop(['CSI','SK_ID'],axis=1)
y = data['CSI']
corr = []
skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=0)
#     ra_boost = []
#     counter = 0
#     param_grid = {'n_estimaros':[100,40,20]}
ar = []
for train_index, test_index in skf.split(X, y):



    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y[train_index], y[test_index]
#        rocauc = roc_auc_score(pred,y_test)
#        print(rocauc)
    model.fit(X_train,y_train, epochs=30, batch_size=64)
    pred = model.predict_proba(X_test)
    ar.append(roc_auc_score(y_test,pred))
#     predictions = model_boost.predict(X_site)
#        print('{} - TEST, {} - correlation'.format(roc_auc_score(y_test,pred),roc_auc_score(best,predictions)))
#    print('{} - correlation'.format(roc_auc_score(best,predictions)))
#     corr.append(roc_auc_score(best,predictions))

Epoch 1/30
3166/3166 [==============================] - 0s 35us/step - loss: 0.4322
Epoch 2/30
3166/3166 [==============================] - 0s 43us/step - loss: 0.4341
Epoch 3/30
3166/3166 [==============================] - 0s 32us/step - loss: 0.4317
Epoch 4/30
3166/3166 [==============================] - 0s 34us/step - loss: 0.4349
Epoch 5/30
3166/3166 [==============================] - 0s 45us/step - loss: 0.4309
Epoch 6/30
3166/3166 [==============================] - 0s 34us/step - loss: 0.4336
Epoch 7/30
3166/3166 [==============================] - 0s 34us/step - loss: 0.4353
Epoch 8/30
3166/3166 [==============================] - 0s 37us/step - loss: 0.4335
Epoch 9/30
3166/3166 [==============================] - 0s 45us/step - loss: 0.4326
Epoch 10/30
3166/3166 [==============================] - 0s 46us/step - loss: 0.4322
Epoch 11/30
3166/3166 [==============================] - 0s 41us/step - loss: 0.4333
Epoch 12/30
3166/3166 [==============================] - 0s 53us/step - lo

In [190]:
sum(ar)/4

0.5554989336257565

In [48]:
def test_model(data, model):

#     param = {'objective':'binary:logistic', 'n_estimators':20}

#     model_boost = xgb.XGBModel(**param)
# Or you can use: clf = xgb.XGBClassifier(**param_dist)
    X = data.drop(['CSI','SK_ID'],axis=1)
    y = data['CSI']
    corr = []
    skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=0)
#     ra_boost = []
#     counter = 0
#     param_grid = {'n_estimaros':[100,40,20]}
    for train_index, test_index in skf.split(X, y):

     
        
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = y[train_index], y[test_index]
#        rocauc = roc_auc_score(pred,y_test)
#        print(rocauc)
        model.fit(X_train,y_train)
        pred = model_boost.predict(X_test)
        ra_boost.append(roc_auc_score(y_test,pred))
        predictions = model_boost.predict(X_site)
#        print('{} - TEST, {} - correlation'.format(roc_auc_score(y_test,pred),roc_auc_score(best,predictions)))
#    print('{} - correlation'.format(roc_auc_score(best,predictions)))
    corr.append(roc_auc_score(best,predictions))
    #print(ra_boost)
#    print('{} - average rocauc'.format(sum(ra_boost, 0.0) / len(ra_boost)))
#    plot_roc_auc(y_test,model_boost.predict(X_test))
#    print(model_boost.feature_importances_)
    print(sum(corr, 0.0)/ len(corr))
    return model_boost

ValueError: Error when checking input: expected dense_1_input to have shape (4,) but got array with shape (7,)

# Подготовка решениия

In [16]:
def get_test_data(path):
    return '../data/test/' + path

In [ ]:
test_data_base = pd.read_csv(get_test_data('subs_csi_test.csv'),sep=';')
test_data_base = test_data_base.drop(['CONTACT_DATE'],axis=1)
test_data_base.head()

In [17]:
test_data_cons = pd.read_csv(get_test_data('subs_bs_consumption_test.csv'),sep=';')

In [21]:
test_data_cons['MON'].unique()

array([1.02, 1.04, 1.03, 1.05])

In [24]:
test_data_cons['CELL_LAC_ID'].unique().shape

(353797,)

In [ ]:
test_data_cons= list_to_float(test_data_cons, ['SUM_MINUTES','SUM_DATA_MB','SUM_DATA_MIN'])
test_data_cons = test_data_cons.groupby(['CELL_LAC_ID','SK_ID']).agg({'SUM_MINUTES':np.sum,
                                                 'SUM_DATA_MB':np.sum,
                                                 'SUM_DATA_MIN':np.sum})

In [45]:
test_data_cons = test_data_cons.fillna(0)
test_data_cons = test_data_cons.reset_index(level=['CELL_LAC_ID', 'SK_ID'])
test_data_cons['SPEED'] = test_data_cons.apply(lambda row: transoform(row),axis=1)
test_data_cons = test_data_cons.groupby(['SK_ID']).agg({'SUM_MINUTES':np.sum,
                                                 'SUM_DATA_MB':np.sum,
                                                 'SUM_DATA_MIN':np.sum,'SPEED':np.mean})

test_data = pd.merge(test_data_base,test_data_cons,on='SK_ID',how='left')

'''
test_data = pd.merge(test_data,data_cell, on='CELL_LAC_ID',how='left')
test_data = test_data.fillna(-100)
test_data = test_data.groupby(['SK_ID']).agg({'NUM':np.mean,
                                                 'SPEED':np.mean,
                                                 'SUM_DATA_MIN':np.sum})
test_data = test_data.reset_index(level=['SK_ID'])
test_data = pd.merge(test_data,test_data_base,on='SK_ID',how='left')


test_data_sub_feat = pd.read_csv('subs_features_test.csv',sep=';')

test_data_sub_feat = test_data_sub_feat.drop(['COM_CAT#24'],axis=1)

list_float = ['COM_CAT#'+str(i) for i in [19,20,21,22,23,31,32,33,27,28,29,30,18,17]]
list_float.extend(['COST','REVENUE','ITC','VAS','RENT_CHANNEL','ROAM'])

test_data_sub_feat = list_to_float(test_data_sub_feat, list_float)

test_data_sub_feat = test_data_sub_feat.groupby(['SNAP_DATE','SK_ID']).agg({'COM_CAT#20':np.sum,
                                                 'COM_CAT#22':np.sum,
                                                 'COM_CAT#23':np.sum,
                                                 
                                                                                           
                                                 'COM_CAT#31':np.sum,
                                                 
                                                 'COM_CAT#33':np.sum,
                                          
                                          'COM_CAT#1':np.sum,
                                       
                                          'COM_CAT#7':np.sum,
                             
                                          'COM_CAT#8':np.sum,
                                         'COM_CAT#20':np.sum,
                                            'ROAM':np.sum,
                                          
                                          'INTERNET_TYPE_ID':np.mean})
                                                
test_data_sub_feat = test_data_sub_feat.reset_index(level=['SK_ID','SNAP_DATE'])
test_data_sub_feat = test_data_sub_feat[test_data_sub_feat['SNAP_DATE'] == max(test_data_sub_feat['SNAP_DATE'])]
test_data_sub_feat = test_data_sub_feat.drop(['SNAP_DATE'],axis=1)

gen_dat_feat = pd.merge(data_base, data_sub_feat,on='SK_ID',how='left')
gen_dat_feat = gen_dat_feat.fillna(calculate_means(gen_dat_feat))
'''

test_data_session = pd.read_csv(get_test_data('subs_bs_data_session_test.csv'),sep=';')
test_data_session['DATA_VOL_MB'] = to_float(test_data_session, 'DATA_VOL_MB')
test_data_session = test_data_session.groupby(['CELL_LAC_ID','SK_ID']).agg({'DATA_VOL_MB': np.sum})
test_data_session = test_data_session.reset_index(level=['CELL_LAC_ID', 'SK_ID'])
test_data_session = test_data_session.groupby('SK_ID').agg({'DATA_VOL_MB':np.mean})
test_data_session = test_data_session.reset_index(level=['SK_ID'])



test_data = pd.merge(test_data,test_data_session,on='SK_ID',how='left')

X_site = test_data.drop(['SK_ID'],axis=1).fillna(0).values
# test_data

KeyboardInterrupt: 

In [137]:
model = test_xgboost(gen_dat)

0.5684129935469758
0.5091463414634146
0.5396101940817388
0.5622142837589992


In [136]:
def test_xgboost(data):

    param = {'objective':'binary:logistic', 'n_estimators':20}

    model_boost = xgb.XGBModel(**param)
# Or you can use: clf = xgb.XGBClassifier(**param_dist)
    X = data.drop(['CSI','SK_ID'],axis=1)
    y = data['CSI']
    corr = []
    skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=0)
    ra_boost = []
    counter = 0
    param_grid = {'n_estimaros':[100,40,20]}
    for train_index, test_index in skf.split(X, y):

     
        
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = y[train_index], y[test_index]
#        rocauc = roc_auc_score(pred,y_test)
#        print(rocauc)
        model_boost.fit(X_train,y_train,eval_metric='auc',verbose=True)
        pred = model_boost.predict(X_test)
        _ = roc_auc_score(y_test,pred)
        print(_)
        ra_boost.append(_)
        predictions = model_boost.predict(X_site)
#        print('{} - TEST, {} - correlation'.format(roc_auc_score(y_test,pred),roc_auc_score(best,predictions)))
#    print('{} - correlation'.format(roc_auc_score(best,predictions)))
#     corr.append(roc_auc_score(best,predictions))
    #print(ra_boost)
#    print('{} - average rocauc'.format(sum(ra_boost, 0.0) / len(ra_boost)))
#    plot_roc_auc(y_test,model_boost.predict(X_test))
#    print(model_boost.feature_importances_)
#     print(sum(corr, 0.0)/ len(corr))
    return model_boost

In [191]:
#X_test_site = test_data.drop(['SK_ID','NUM'],axis=1).fillna(0)
predictions = model.predict(X_site)
#predictions = model_boost.predict(X_test_site.values)
with open('1.csv','w') as fout:
    for el in predictions:
        fout.write(str(el[0])+'\n')
# best = pd.read_csv('best53.csv')
# best = np.vstack((np.array(0),np.array(best)))
# roc_auc_score(,predictions)

In [192]:
sum((predictions > 0.2))

array([125])

In [155]:
with open('3.csv','w') as fout:
    for i in range(len(best54)):
        temp = float(best54[i])/2+float(current[i])/2
        temp += float(random[i])/2
        fout.write(str(temp)+'\n')

TypeError: plot_roc_auc() missing 1 required positional argument: 'y_test'

### Проверка с лушчим решением


In [193]:
best54 = pd.read_csv('3.csv')
current = pd.read_csv('1.csv')
# random = pd.read_csv('best53.csv')

best54 = np.vstack((np.array(0.24409929),np.array(best54)))
current = np.vstack((np.array(0.18459004),np.array(current)))
# random = np.vstack((np.array(0),np.array(random)))

with open('4.csv','w') as fout:
    for i in range(len(best54)):
        temp = float(best54[i]) * 0.2+float(current[i])*0.8
#         temp += float(random[i])/2
        fout.write(str(temp)+'\n')
# new = pd.read_csv('best53.csv')
# new = np.vstack((np.array(0),np.array(new)))
# roc_auc_score(best,new)


In [86]:
X

array([[1.04033e-01, 1.95380e-02, 2.00000e+00, ..., 1.36000e+02,
        4.00000e+00, 7.01257e-01],
       [1.03896e-01, 5.33110e-02, 1.00000e+00, ..., 1.49000e+02,
        3.00000e+00, 7.04834e-01],
       [1.03896e-01, 6.89530e-02, 3.00000e+00, ..., 1.27700e+03,
        6.00000e+00, 6.99904e-01],
       ...,
       [1.03896e-01, 1.13040e-02, 4.00000e+00, ..., 0.00000e+00,
        5.00000e+00, 6.98539e-01],
       [1.04275e-01, 7.56940e-02, 1.00000e+00, ..., 7.95000e+02,
        1.00000e+00, 7.14844e-01],
       [1.03896e-01, 3.15000e-04, 3.00000e+00, ..., 1.32400e+03,
        4.00000e+00, 6.98463e-01]])

In [87]:
test_data

,SK_ID,COM_CAT#33,COM_CAT#31,COM_CAT#7,INTERNET_TYPE_ID,COM_CAT#23,COM_CAT#20,ROAM,COM_CAT#8,COM_CAT#1,COM_CAT#22
0,6184,0.104033,0.019538,2.0,2.0,0.118528,0.701257,0.0,136.0,4.0,0.701257
1,5175,0.103896,0.053311,1.0,2.0,0.130575,0.704834,0.0,149.0,3.0,0.704834
2,5478,0.103896,0.068953,3.0,2.0,0.112392,0.699904,0.0,1277.0,6.0,0.699904
3,7807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5640,0.103896,0.081004,1.0,2.0,0.140597,0.706550,0.0,1279.0,8.0,0.706550
5,6083,0.103896,0.000704,5.0,3.0,0.109409,0.698418,0.0,1249.0,7.0,0.698418
6,2707,0.103896,0.047827,3.0,1.0,0.134624,0.706688,0.0,1575.0,5.0,0.706688
7,6681,0.103896,0.000841,1.0,1.0,0.109409,0.698453,0.0,1645.0,8.0,0.698453
8,8976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9526,0.103896,0.077593,2.0,1.0,0.122253,0.704759,0.0,0.0,7.0,0.704759
